import pandas as pd

df = pd.read_csv("/kaggle/input/news-recom-dataset/processed_articles.csv)
df.head()

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np



In [3]:
import pandas as pd

df = pd.read_csv("/kaggle/input/newsapi-popularsources/news_dataset_new.csv", sep = '|', engine = "python", encoding = 'utf-8' )
df.head()

,uri,title,date,url,body,source_uri,source_title,image,category
0,2024-03-285371454,"$1,000 Disrupt pass savings end Friday | TechC...",2024-03-11,https://techcrunch.com/2024/03/11/1000-disrupt...,"TechCrunch Disrupt, the premier early-stage st...",techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology
1,2024-03-294655317,"Watch: EV makers are struggling, and Fisker's ...",2024-03-19,https://techcrunch.com/2024/03/19/techcrunch-m...,Fisker's finances are back in the news after t...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology
2,8023324186,Worldcoin fails to get injunction against Spai...,2024-03-11,https://techcrunch.com/2024/03/11/worldcoin-fa...,Controversial eyeball scanning startup Worldco...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology
3,2024-03-285331789,Bluesky is funding developer projects to give ...,2024-03-11,https://techcrunch.com/2024/03/11/bluesky-is-f...,Would-be Twitter/X rival Bluesky is looking to...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology
4,2024-03-299447780,Mallard Bay is the Airbnb for guided hunting a...,2024-03-23,https://techcrunch.com/2024/03/23/deal-dive-bu...,Americans spent more than $144.8 billion on fi...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology


In [4]:
# Remove rows where the 'uri' is duplicated since 'uri' is a unique identifier for the articles
df = df.drop_duplicates(subset=['uri','title'])
df.describe

<bound method NDFrame.describe of                      uri                                              title  \
0      2024-03-285371454  $1,000 Disrupt pass savings end Friday | TechC...   
1      2024-03-294655317  Watch: EV makers are struggling, and Fisker's ...   
2             8023324186  Worldcoin fails to get injunction against Spai...   
3      2024-03-285331789  Bluesky is funding developer projects to give ...   
4      2024-03-299447780  Mallard Bay is the Airbnb for guided hunting a...   
...                  ...                                                ...   
23659  2024-03-300544732  Pandemic accord talks at loggerheads as time t...   
23660         8045736315        Bygone brand: How expat fam lost its status   
23661         8045710985                      When do you plan to be happy?   
23662         8030598844  Drug-free options for managing migraine in ado...   
23663         8045596349  Civic Body Launches Anti-mosquito Drive | Patn...   

             date

In [5]:
# Drop rows with missing 'title', 'body', or 'category'
df = df.dropna()



In [6]:
df.describe

<bound method NDFrame.describe of                      uri                                              title  \
0      2024-03-285371454  $1,000 Disrupt pass savings end Friday | TechC...   
1      2024-03-294655317  Watch: EV makers are struggling, and Fisker's ...   
2             8023324186  Worldcoin fails to get injunction against Spai...   
3      2024-03-285331789  Bluesky is funding developer projects to give ...   
4      2024-03-299447780  Mallard Bay is the Airbnb for guided hunting a...   
...                  ...                                                ...   
23659  2024-03-300544732  Pandemic accord talks at loggerheads as time t...   
23660         8045736315        Bygone brand: How expat fam lost its status   
23661         8045710985                      When do you plan to be happy?   
23662         8030598844  Drug-free options for managing migraine in ado...   
23663         8045596349  Civic Body Launches Anti-mosquito Drive | Patn...   

             date

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Download stopwords from NLTK
import nltk
nltk.download('stopwords')

# Initialize stemmer
stemmer = PorterStemmer()

# Load stopwords list once
english_stopwords = set(stopwords.words('english'))
# Assume df is your initial DataFrame
df_cleaned = df.copy() 




# Optimized text cleaning function
def clean_text_optimized(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in english_stopwords]
    return ' '.join(words)

# Use parallel processing (may require additional setup)
from multiprocessing import Pool

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Wrap your text cleaning in another function to use with parallelization
def clean_text_on_df(df):
    # Apply the cleaning function to 'title' and 'body' using .loc for stable assignment
    df.loc[:, 'cleaned_title'] = df['title'].apply(clean_text_optimized)
    df.loc[:, 'cleaned_body'] = df['body'].apply(clean_text_optimized)
    return df




[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Assuming num_partitions is the number of partitions to split dataframe
# And num_cores is the number of cores on your machine
num_partitions = 1000  # number of partitions to split dataframe
num_cores = 4  # number of cores on your machine

df_cleaned = parallelize_dataframe(df_cleaned, clean_text_on_df)


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [10]:
df_cleaned.head()

,uri,title,date,url,body,source_uri,source_title,image,category,cleaned_title,cleaned_body
0,2024-03-285371454,"$1,000 Disrupt pass savings end Friday | TechC...",2024-03-11,https://techcrunch.com/2024/03/11/1000-disrupt...,"TechCrunch Disrupt, the premier early-stage st...",techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology,1 000 disrupt pass save end friday techcrunch,techcrunch disrupt premier earli stage startup...
1,2024-03-294655317,"Watch: EV makers are struggling, and Fisker's ...",2024-03-19,https://techcrunch.com/2024/03/19/techcrunch-m...,Fisker's finances are back in the news after t...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology,watch ev maker struggl fisker particularli pre...,fisker financ back news compani warn back febr...
2,8023324186,Worldcoin fails to get injunction against Spai...,2024-03-11,https://techcrunch.com/2024/03/11/worldcoin-fa...,Controversial eyeball scanning startup Worldco...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology,worldcoin fail get injunct spain privaci suspe...,controversi eyebal scan startup worldcoin fail...
3,2024-03-285331789,Bluesky is funding developer projects to give ...,2024-03-11,https://techcrunch.com/2024/03/11/bluesky-is-f...,Would-be Twitter/X rival Bluesky is looking to...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology,blueski fund develop project give twitter x al...,would twitter x rival blueski look directli in...
4,2024-03-299447780,Mallard Bay is the Airbnb for guided hunting a...,2024-03-23,https://techcrunch.com/2024/03/23/deal-dive-bu...,Americans spent more than $144.8 billion on fi...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology,mallard bay airbnb guid hunt fish techcrunch,american spent 144 8 billion fish hunt 2022 al...


In [11]:
df.head()

,uri,title,date,url,body,source_uri,source_title,image,category
0,2024-03-285371454,"$1,000 Disrupt pass savings end Friday | TechC...",2024-03-11,https://techcrunch.com/2024/03/11/1000-disrupt...,"TechCrunch Disrupt, the premier early-stage st...",techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology
1,2024-03-294655317,"Watch: EV makers are struggling, and Fisker's ...",2024-03-19,https://techcrunch.com/2024/03/19/techcrunch-m...,Fisker's finances are back in the news after t...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology
2,8023324186,Worldcoin fails to get injunction against Spai...,2024-03-11,https://techcrunch.com/2024/03/11/worldcoin-fa...,Controversial eyeball scanning startup Worldco...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology
3,2024-03-285331789,Bluesky is funding developer projects to give ...,2024-03-11,https://techcrunch.com/2024/03/11/bluesky-is-f...,Would-be Twitter/X rival Bluesky is looking to...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2023...,Technology
4,2024-03-299447780,Mallard Bay is the Airbnb for guided hunting a...,2024-03-23,https://techcrunch.com/2024/03/23/deal-dive-bu...,Americans spent more than $144.8 billion on fi...,techcrunch.com,TechCrunch,https://techcrunch.com/wp-content/uploads/2024...,Technology


In [12]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=1000)  # limiting to 1000 features for simplicity

# Fit and transform the 'cleaned_title' field
tfidf_matrix = tfidf.fit_transform(df_cleaned['cleaned_title'])


In [13]:
# Compute cosine similarity for the articles
article_similarities = cosine_similarity(tfidf_matrix)

# You'd have to decide on an appropriate threshold for considering articles to be duplicates
similarity_threshold = 0.87 # for example

# Iterate over the similarity matrix and mark articles for removal
to_remove = []
for i in range(article_similarities.shape[0]):
    for j in range(i+1, article_similarities.shape[1]):
        if article_similarities[i, j] > similarity_threshold:
            to_remove.append(j)

# Deduplicate
df_cleaned = df_cleaned.drop(df_cleaned.index[to_remove])

In [14]:
df_cleaned.reset_index(drop=True, inplace=True)


In [15]:
df_cleaned["category"].unique()

array(['Technology', 'Sports', 'Politics', 'Science', 'Business',
       'Health'], dtype=object)

In [16]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=1000)  # limiting to 1000 features for simplicity

# Fit and transform the 'cleaned_title' field
tfidf_matrix = tfidf.fit_transform(df_cleaned['cleaned_title'])

In [17]:


def get_article_index_by_uri(uri, df):
    # Ensure the URI exists in the DataFrame to prevent IndexError
    if uri in df['uri'].values:
        return df.index[df['uri'] == uri].tolist()[0]
    return None

def create_initial_user_profile(category_preferences, df, tfidf_matrix):
    # Identify articles that match the user's category preferences
    articles_indices = df[df['category'].isin(category_preferences)].index
    # Create the user profile by averaging the TF-IDF vectors of these articles
    user_profile_vector = tfidf_matrix[articles_indices].mean(axis=0)
    return user_profile_vector

# Define a function to recommend articles based on the user profile vector
def recommend_articles(user_profile_vector, tfidf_matrix, df_articles, top_n=10):
    # Calculate the cosine similarity between the user profile and all article vectors
    cosine_similarities = cosine_similarity(user_profile_vector, tfidf_matrix)
    # Get the indices of articles with the highest similarity scores
    top_article_indices = np.argsort(cosine_similarities.flatten())[-top_n:][::-1]
    # Return the recommended articles
    recommended_articles = df_articles.iloc[top_article_indices]
    return recommended_articles

def recommend_for_new_user(user_preferences, df, tfidf_matrix, top_n=10):
    # Simulate user profile creation based on their selected categories
    user_profile_vector = create_initial_user_profile(user_preferences, df, tfidf_matrix)
    
    if isinstance(user_profile_vector, csr_matrix):
        user_profile_vector = user_profile_vector.toarray()
    user_profile_vector = np.asarray(user_profile_vector).reshape(1, -1)  # Ensure it's 2D
    
    recommended_articles_df = recommend_articles(user_profile_vector, tfidf_matrix, df, top_n)
    return recommended_articles_df




def recommend_based_on_clicks(user_clicks_uris, df, tfidf_matrix, top_n=10):
    """Recommend articles based on articles a user has clicked on."""
    clicked_indices = [get_article_index_by_uri(uri, df) for uri in user_clicks_uris]
    
    # Ensure only valid indices are used
    clicked_indices = [idx for idx in clicked_indices if idx is not None and idx < tfidf_matrix.shape[0]]
    
    if not clicked_indices:
        # Return an empty DataFrame or a default recommendation if no valid clicks
        return pd.DataFrame()
    
    # Aggregate TF-IDF vectors of clicked articles
    user_profile_vector = tfidf_matrix[clicked_indices].mean(axis=0)
    
    # Assuming user_profile_vector might not be properly reshaped after toarray()
    if isinstance(user_profile_vector, csr_matrix):
        user_profile_vector = user_profile_vector.toarray()
    user_profile_vector = np.asarray(user_profile_vector).reshape(1, -1)  # Ensure it's 2D

    # Proceed to compute cosine similarity
    similarities = cosine_similarity(user_profile_vector, tfidf_matrix)


    
    # Get top N recommendations, excluding already clicked articles
    recommended_indices = similarities.argsort()[0][-top_n-len(clicked_indices):][::-1]
    recommended_indices = [idx for idx in recommended_indices if idx not in clicked_indices][:top_n]
    
    # Return recommended articles
    return df.iloc[recommended_indices]




In [18]:
technology_articles = df_cleaned[df_cleaned['category'].isin(['Health',"Computers","Sports" ])]

# Sample 5 random URIs from the 'Technology' category
sampled_uris = technology_articles['uri'].sample(15).tolist()
print(sampled_uris)


# Generate recommendations based on clicks
recommended_articles = recommend_based_on_clicks(sampled_uris, df_cleaned, tfidf_matrix, top_n=10)
print(recommended_articles[['uri', 'title']])

['8069096014', '8045812059', '8067022230', '8055663444', '8046975651', '8023611045', '8039713159', '8053720334', '8043243246', '2024-03-291607066', '8023410422', '8064476294', '2024-03-289200578', '8054412673', '8023497587']
                     uri                                              title
11887  2024-03-299405783  Public Health Foundation of India ranked secon...
14483         8065155467  Dr Mansukh Mandaviya reviews public health pre...
6001          8065123165  Derrick Brown, Panthers agree to four-year ext...
4224          8041732350  CSK vs RCB Live Score, IPL 2024: Dhoni's new r...
4873          8023986634  Report: Damien Lewis has agreed to a four-year...
12397         8044377675  'The Invisible Shield' Aims To Makes Public He...
7955          8070681467  As temperature soars, T.N. Public Health Direc...
2960   2024-03-307265247  GT vs SRH Live Score, IPL 2024: Misfiring Guja...
4145          8043029630  Virat Kohli hurls a mouthful in X-rated send-o...
14040         8

In [19]:
# Example of new user preferences (selected during onboarding)
new_user_preferences = ['Business', 'Health']

# Get recommendations for the new user
recommendations_for_new_user = recommend_for_new_user(new_user_preferences, df_cleaned, tfidf_matrix, top_n=10)
print(recommendations_for_new_user[['uri', 'title']])


              uri                                              title
14043  8070548811  Lacto-vegetarian or ovo-vegetarian: What's you...
14437  8023972326  The unbearable shyness of peeing | India News ...
14301  8020857613  The unbearable shyness of peeing | Ahmedabad N...
14377  8066672634  Cancer, Horoscope Today, April 7, 2024: Day to...
9320   8062334488  Solar Eclipse 2024: Date, Time, Sutak Timing o...
14491  8068225610  Docs' March On World Health Day | Lucknow News...
9353   8036574950  Nisar only in 2nd half of 2024; Trishna gets n...
14546  8042996492  Bhu Docs Study Molecular Aspects Of Testicular...
14545  8032569087  Rehabilitation in Mental Health: Urgent Need f...
14246  8065767131  World Health Day 2024 - The Wisdom of Self-Ref...


In [20]:
from scipy.sparse import save_npz

# Save the TF-IDF matrix
save_npz('/kaggle/working/tfidf_matrix_popular.npz', tfidf_matrix)

# Save the DataFrame
df_cleaned.to_pickle('/kaggle/working/df_cleaned_popular.pkl')
